In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]])
target = torch.tensor([[0],[0],[1],[1.]])

# A Toy Model
model = nn.Linear(2,1)  # class torch.nn.Linear(in_features, out_features, bias=True), y = ax + b

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)

In [4]:
train()

tensor(2.0864)
tensor(0.1730)
tensor(0.0232)
tensor(0.0082)
tensor(0.0048)
tensor(0.0030)
tensor(0.0019)
tensor(0.0012)
tensor(0.0008)
tensor(0.0005)
tensor(0.0003)
tensor(0.0002)
tensor(0.0001)
tensor(8.6628e-05)
tensor(5.5720e-05)
tensor(3.5872e-05)
tensor(2.3120e-05)
tensor(1.4921e-05)
tensor(9.6441e-06)
tensor(6.2444e-06)


In [5]:
import syft as sy
hook = sy.TorchHook(torch)

In [6]:
# create a couple workers

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [7]:
# A Toy Dataset
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

In [8]:
from syft.federated.floptimizer import Optims
workers = ['bob', 'alice']
optims = Optims(workers, optim=optim.Adam(params=model.parameters(),lr=0.1))

In [9]:
def train():
    # Training Logic
    for iter in range(10):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)
            
            #Call the optimizer for the worker using get_optim
            opt = optims.get_optim(data.location.id)
            #print(data.location.id)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much we missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # 5) change those weights
            opt.step()
            
            # NEW) get model (with gradients)
            model.get()

            # 6) print our progress
            print(loss.get().data) # NEW) slight edit... need to call .get() on loss\
    
# federated averaging

In [10]:
train()

tensor(0.6517)
tensor(7.5636)
tensor(0.1334)
tensor(4.8146)
tensor(0.0114)
tensor(2.9042)
tensor(0.1912)
tensor(1.7422)
tensor(0.4787)
tensor(1.0661)
tensor(0.7293)
tensor(0.6649)
tensor(0.9013)
tensor(0.4222)
tensor(0.9940)
tensor(0.2743)
tensor(1.0173)
tensor(0.1841)
tensor(0.9831)
tensor(0.1289)
